In [ ]:
# config

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import yaml

In [2]:
import sys
sys.path.append("..")

from intlib.integrators import RK3Integrator, RKLikeMatrixIntegrator
from intlib.targets import ScalarLinearTarget
from intlib.trainers import NoGroundTruthMSETaylorTrainer
from intlib.taylorloss import MSETaylorLossFunc

In [3]:
config = yaml.safe_load(open('config.yml'))
cfg = config['linear']

In [4]:
dim = cfg['dim']
target = ScalarLinearTarget(dim=dim) # different targets in our experiments only require some changes here

In [5]:
rk3 = RK3Integrator(target=target, dim=dim)

In [6]:
integrator_stage = cfg['integrator_stage']
taylorloss_order = cfg['taylorloss_order']
inte_seed = cfg['inte_seed']
data_seed = cfg['data_seed']

In [7]:
tf.random.set_seed(inte_seed)
rk_nn = RKLikeMatrixIntegrator(target=target, dim=dim, order=integrator_stage)

In [8]:
# Construct the trainer with MSE + Taylorseries-based Regularizer
MSETaylorloss = MSETaylorLossFunc(order=taylorloss_order).calc_loss

In [9]:
trainer = NoGroundTruthMSETaylorTrainer(integrator=rk_nn, 
                                        refer_integrator=rk3,
                                        target=target,
                                        loss_func=MSETaylorloss,
                                        loss_order=taylorloss_order,
                                        step=cfg['step'])

In [10]:
# Generate data 
tf.random.set_seed(data_seed)
train_data = trainer.generate_data(train_size=3000)

In [11]:
# Train the model, gamma is the weight of MSE and mu is that of Taylorseries-based Regularizer. 
# 'ratio' is an observation that shows 'error_rk / error_nn', we want to train is to below 1. 'ratio_lim' is an early_stopping index.
epochs, current_loss, mse, taylorloss = trainer.train(train_data = train_data,
                                                      epochs=cfg['epochs'],
                                                      learning_rate=cfg['lr'],
                                                      gamma_init=cfg['gamma_init'],
                                                      mu_init=cfg['mu_init'],
                                                      ratio_lim=cfg['ratio_lim'],
                                                      coefficient_weight=cfg['coefficient_weight'],
                                                      lr_decay=cfg['lr_decay'])

ratio 135270608.8466124
squ_taylorloss_init 2.0623346953157693
gamma 1.0
0 1.0311673611849454e+16
lr 0.1
ratio 90439935.70692055
squ_taylorloss_init 1.6610308425772546
gamma 1.0
1 8305154303326209.0
lr 0.1
ratio 52634915.91149719
squ_taylorloss_init 1.213525947199921
gamma 1.0
2 6067629788634521.0
lr 0.1
ratio 22503449.751531996
squ_taylorloss_init 0.7541292239351973
gamma 1.0
3 3770646142179436.5
lr 0.1
ratio 3886605.131495467
squ_taylorloss_init 0.34011926250907615
gamma 1.0
4 1700596316431986.0
lr 0.1
ratio 1023666.125959653
squ_taylorloss_init 0.07545771760987205
gamma 1.0
5 377288589073026.4
lr 0.1
ratio 16001971.014857799
squ_taylorloss_init 0.044951544331382466
gamma 1.0
6 224757737658883.34
lr 0.1
ratio 41930495.682558134
squ_taylorloss_init 0.1534816663184583
gamma 1.0
7 767408373522787.1
lr 0.1
ratio 63782473.99215322
squ_taylorloss_init 0.25128916159696085
gamma 1.0
8 1256445871767278.2
lr 0.1
ratio 72958274.2549316
squ_taylorloss_init 0.2924392537160917
gamma 1.0
9 14621963

In [14]:
dir(rk_nn)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'build',
 'dim',
 'ks',
 'model',
 'order',
 'set_rk2_weights',
 'set_rk3_weights',
 'set_rk4_weights',
 'step',
 'target']

In [15]:
dir(trainer)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'build',
 'current_loss',
 'current_mse',
 'current_ratio',
 'current_taylorloss',
 'gamma',
 'generate_data',
 'generate_fixed_inputs',
 'generate_inputs',
 'integrator',
 'loss_func',
 'loss_order',
 'lr',
 'model',
 'mu',
 'optimizer',
 'refer_integrator',
 'sample_h0s',
 'sample_hs',
 'step',
 'target',
 'train']

In [ ]:
type(rk3)